# Pivot Tables in SQL

Now, let's try to accomplish the same task and make a pivot table in SQL. 

There are a couple different ways to accomplish this since there are various flavours of SQL depending on the relational database managment system (RDBMS) that you are using. Each RDBMS supports different methods which can be applied to achieve a similar effect, but some are easier than others. For instance, Oracle and SQL Server have `pivot` and `unipivot` clauses while PostgreSQL has the `crosstab` clause which are specific for making pivot tables.  

For this example, we'll use SQLite because it's free and publicly-available. Also, there are a couple of different options for making it work within a Jupyter notebook. Below, I use [Catherine Devlin's %sql-magic for IPython](https://github.com/catherinedevlin/ipython-sql) to connect to my SQLite database. However, because SQLite doesn't have a built-in pivot clause, we'll have to do a bit more work. I'll also discuss some other limitations as we go along. 

### Connect to the database

In [4]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [2]:
%sql sqlite:///example.db

'Connected: None@example.db'

### Import the data

The command line interface (CLI) commands to import a CSV file doesn't work in Jupyter unfortunately, so instead, I ran the following code from CLI. 

```
.mode csv
.separator ","
.import nhex-fixed.csv nhex
```

Check to to see that the data was imported correctly:

In [9]:
%%sql
SELECT * FROM nhex LIMIT 5;

Done.


,Year,Jurisdiction,Sector,Use of funds,Age group,Gender,Dollars per capita
60480,2014,Newfoundland and Labrador,Provincial/territorial government,Total,0,F,10762.0
60481,2014,Newfoundland and Labrador,Provincial/territorial government,Total,1,F,1708.0
60482,2014,Newfoundland and Labrador,Provincial/territorial government,Total,5,F,1642.0
60483,2014,Newfoundland and Labrador,Provincial/territorial government,Total,10,F,1568.0
60484,2014,Newfoundland and Labrador,Provincial/territorial government,Total,15,F,2187.0


Now that our data has been imported, we can begin creating our pivot table!

## Pivoting in SQLite

Since we do not have a built-in `pivot` clause, we will manually combine some aggregate functions with filtering and grouping to achieve the task.

1. **Filter** - I have already filtered the dataset for the year 2014, but if I had yet to do this, I could use a `WHERE` clause (e.g. `WHERE Year=2014`)
2. **Column labels** - Each label must be indicated in the `SELECT` clause including the header for the row labels (i.e. Jurisdiction an Use of funds). Each Age Group label is selected using a `CASE` statment. `SUM(CASE WHEN "Age Group"=0 THEN "Dollars per capita" END) AS "<1"`
3. **Row labels** - `GROUP BY Jurisdiction, "Use of funds";`
4. **Summation values** - This is specified within the `SUM` function and the `CASE` statment. See #2 above. 


In [40]:
%%sql
SELECT Jurisdiction, "Use of funds"
    , SUM(CASE WHEN "Age Group"=0 THEN "Dollars per capita" END) AS "<1"
    , SUM(CASE WHEN "Age Group"=1 THEN "Dollars per capita" END) AS "1-4"
    , SUM(CASE WHEN "Age Group"=5 THEN "Dollars per capita" END) AS "5-9"
    , SUM(CASE WHEN "Age Group"=10 THEN "Dollars per capita" END) AS "10-14"
    , SUM(CASE WHEN "Age Group"=15 THEN "Dollars per capita" END) AS "15-19"
    , SUM(CASE WHEN "Age Group"=20 THEN "Dollars per capita" END) AS "20-24"
    , SUM(CASE WHEN "Age Group"=25 THEN "Dollars per capita" END) AS "25-29"
    , SUM(CASE WHEN "Age Group"=30 THEN "Dollars per capita" END) AS "30-34"
    , SUM(CASE WHEN "Age Group"=35 THEN "Dollars per capita" END) AS "35-39"
    , SUM(CASE WHEN "Age Group"=40 THEN "Dollars per capita" END) AS "40-44"
    , SUM(CASE WHEN "Age Group"=45 THEN "Dollars per capita" END) AS "45-49"
    , SUM(CASE WHEN "Age Group"=50 THEN "Dollars per capita" END) AS "50-54"
    , SUM(CASE WHEN "Age Group"=55 THEN "Dollars per capita" END) AS "55-59"
    , SUM(CASE WHEN "Age Group"=60 THEN "Dollars per capita" END) AS "60-64"
    , SUM(CASE WHEN "Age Group"=65 THEN "Dollars per capita" END) AS "65-69"
    , SUM(CASE WHEN "Age Group"=70 THEN "Dollars per capita" END) AS "70-74"
    , SUM(CASE WHEN "Age Group"=75 THEN "Dollars per capita" END) AS "75-79"
    , SUM(CASE WHEN "Age Group"=80 THEN "Dollars per capita" END) AS "80-84"
    , SUM(CASE WHEN "Age Group"=85 THEN "Dollars per capita" END) AS "85-89"
    , SUM(CASE WHEN "Age Group"=90 THEN "Dollars per capita" END) AS "90+"
    
FROM nhex
GROUP BY Jurisdiction, "Use of funds";

Done.


Jurisdiction,Use of funds,<1,1-4,5-9,10-14,15-19,20-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85-89,90+
Alberta,Drugs,20.0,30.0,45.0,68.0,87.0,144.0,183.0,227.0,319.0,438.0,510.0,512.0,626.0,963.0,2404.0,3049.0,3493.0,3890.0,4338.0,4316.0
Alberta,Hospitals,24939.0,1059.0,568.0,828.0,1461.0,1641.0,2051.0,2231.0,2066.0,2089.0,2624.0,3466.0,4863.0,6812.0,9260.0,12623.0,18026.0,25132.0,32764.0,42139.0
Alberta,Other institutions,0.0,0.0,0.0,31.0,31.0,22.0,25.0,27.0,27.0,29.0,137.0,181.0,253.0,352.0,883.0,2055.0,4697.0,11240.0,23835.0,30364.0
Alberta,Other professionals,6.0,98.0,291.0,320.0,258.0,87.0,69.0,64.0,78.0,98.0,96.0,87.0,83.0,93.0,132.0,151.0,161.0,168.0,167.0,171.0
Alberta,Physicians,3433.0,1147.0,870.0,852.0,1195.0,1337.0,1598.0,1724.0,1670.0,1690.0,1865.0,2170.0,2510.0,3018.0,3634.0,4453.0,5329.0,6067.0,6590.0,7031.0
Alberta,Total,29748.0,3685.0,3126.0,3450.0,4381.0,4581.0,5276.0,5624.0,5511.0,5696.0,6582.0,7767.0,9686.0,12588.0,17666.0,23681.0,33057.0,47848.0,69045.0,85371.0
British Columbia,Drugs,12.0,22.0,46.0,79.0,108.0,162.0,227.0,271.0,309.0,343.0,402.0,473.0,520.0,557.0,607.0,709.0,1048.0,1233.0,1307.0,1173.0
British Columbia,Hospitals,17501.0,577.0,378.0,549.0,995.0,1119.0,1556.0,1817.0,1724.0,1526.0,1832.0,2435.0,3221.0,4404.0,5917.0,8154.0,11497.0,15346.0,20945.0,26583.0
British Columbia,Other institutions,3.0,4.0,3.0,2.0,12.0,20.0,26.0,29.0,30.0,29.0,69.0,91.0,120.0,165.0,314.0,609.0,1349.0,3040.0,7145.0,9091.0
British Columbia,Other professionals,6.0,59.0,164.0,166.0,119.0,61.0,93.0,117.0,87.0,61.0,52.0,50.0,49.0,54.0,63.0,68.0,74.0,72.0,69.0,60.0


### Limitations

You probably noticed that I had to quite a bit of typing to manually create a column for each age group. There is no automatic way to propogate the columns in SQLite like we did with Excel and Python. However, this difficulty can be overcome using the dynamic pivoting clauses from proprietary RDBMS like Orable and SQL Server. 

## Summary

This sums up pivot tables three ways - Excel, Python, and SQL. Each method has its own advantages and limitations. Of course, there are many other tools that can be used to pivot your data. Choosing between them ultimately depends on your own needs, experience, and format of the data. 
